In [1]:
import os
import numpy as np

DATA_PATH = os.path.join('datasets')

no_sequences = 30
sequence_length = 30
actions = np.array(['hello', 'happy', 'iloveyou'])

In [2]:
label_map = {label:num for num, label in enumerate(actions)}

In [3]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            rep = []
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            for i in res:
                rep.append(np.float32(i))
            window.append(rep)
        sequences.append(window)
        labels.append(label_map[action])

np.array(sequences, dtype=np.float32).shape

(90, 30, 141)

In [4]:
X = np.array(sequences)
labels = np.array(labels)
print(labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import load_model

In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [14]:
label = to_categorical(labels)
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((81, 30, 141), (9, 30, 141), (81, 3), (9, 3))

In [21]:
try:
    os.makedir('models')
except:
    pass

model_path = os.path.join('models', 'model.h5')
    
model = Sequential([
    LSTM(100, activation='LeakyReLU', input_shape=(X_train.shape[1:])),
    BatchNormalization(),
    Dense(75, activation = 'LeakyReLU'),
    BatchNormalization(),
    Dense(50, activation='LeakyReLU'),
    BatchNormalization(),
    Dense(25, activation='LeakyReLU'),
    BatchNormalization(),
    Dense(9, activation='LeakyReLU'),
    BatchNormalization(),
    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(),
              loss='CategoricalCrossentropy',
              metrics=['acc'])

EPOCHS = 80
BATCHSIZE = 8

model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCHSIZE)
model.save(model_path)

Epoch 1/80
11/11 [==============================] - 4s 26ms/step - loss: 1.3920 - acc: 0.3457
Epoch 2/80
11/11 [==============================] - 0s 25ms/step - loss: 1.0379 - acc: 0.4691
Epoch 3/80
11/11 [==============================] - 0s 25ms/step - loss: 1.1635 - acc: 0.5062
Epoch 4/80
11/11 [==============================] - 0s 26ms/step - loss: 1.0687 - acc: 0.5185
Epoch 5/80
11/11 [==============================] - 0s 25ms/step - loss: 0.8966 - acc: 0.5802
Epoch 6/80
11/11 [==============================] - 0s 26ms/step - loss: 1.1428 - acc: 0.3951
Epoch 7/80
11/11 [==============================] - 0s 26ms/step - loss: 1.5552 - acc: 0.3580
Epoch 8/80
11/11 [==============================] - 0s 27ms/step - loss: 1.3855 - acc: 0.2346
Epoch 9/80
11/11 [==============================] - 0s 26ms/step - loss: 1.3049 - acc: 0.3457
Epoch 10/80
11/11 [==============================] - 0s 26ms/step - loss: 1.1711 - acc: 0.4198
Epoch 11/80
11/11 [==============================] - 0s 26m

In [22]:
models = load_model(model_path)
models.evaluate(X_test, y_test)

1/1 [==============================] - 1s 611ms/step - loss: 0.0180 - acc: 1.0000


[0.017973951995372772, 1.0]